In [5]:
import time
from collections import Counter

The file contains the edges of a directed graph. Vertices are labeled as positive integers from 1 to 875714. Every row indicates an edge, the vertex label in first column is the tail and the vertex label in second column is the head (recall the graph is directed, and the edges are directed from the first column vertex to the second column vertex). So for example, the 11th row looks liks : "2 47646". This just means that the vertex with label 2 has an outgoing edge to the vertex with label 47646

Your task is to code up the algorithm from the video lectures for computing strongly connected components (SCCs), and to run this algorithm on the given graph.

Output Format: You should output the sizes of the 5 largest SCCs in the given graph, in decreasing order of sizes, separated by commas (avoid any spaces). So if your algorithm computes the sizes of the five largest SCCs to be 500, 400, 300, 200 and 100, then your answer should be "500,400,300,200,100" (without the quotes). If your algorithm finds less than 5 SCCs, then write 0 for the remaining terms. Thus, if your algorithm computes only 3 SCCs whose sizes are 400, 300, and 100, then your answer should be "400,300,100,0,0" (without the quotes). (Note also that your answer should not have any spaces in it.)

WARNING: This is the most challenging programming assignment of the course. Because of the size of the graph you may have to manage memory carefully. The best way to do this depends on your programming language and environment, and we strongly suggest that you exchange tips for doing this on the discussion forums.


In [20]:
def read_input(filename):
    with open('week5_file/'+filename) as f:
        G = {}
        G_rev = {} #G rev = G with all arcs reversed
        maximum = 0
        for line in f:
            line = line.split() # to deal with blank 
            if line:            # lines (ie skip them)
                line = [int(i) for i in line]
                max_line = max(line)
                if maximum < max_line:
                    maximum = max_line
                u = line[0]
                v = line[1]
                if G.get(u) == None:
                    G[u] = [(u,v)]
                else:
                    G[u].append((u,v))
                if G_rev.get(v) == None:
                    G_rev[v] = [(v,u)]
                else:
                    G_rev[v].append((v,u))
    return G,G_rev,maximum

In [21]:
class Graph:
        
    def __init__(self,n,graph_dict=None):
        if graph_dict == None:
            graph_dict = {}
        self.graph_dict = graph_dict
        self.length = n
        self.explored = [0]*(self.length+1)
        self.leader = [0]*(self.length+1)
        self.finishing_time = [0]*(self.length+1)

In [22]:
def DFS_loop(G):
    global t #finishing time in first pass, number of nodes processed so far
    global s #leaders in second pass
    t = 0
    s = None
    n = G.length
    for i in range(n,0,-1):
        if G.explored[i] == False: #if not yet explored
            s = i
            DFS(G,i)

In [23]:
def DFS(G,i):
    global t
    global s
    stack = [i]
    stack2 = [] #to keep track of the finishing time for iterative dfs
    while len(stack):
        v = stack.pop()
        if G.explored[v] == False:
            G.explored[v] = True
            G.leader[v] = s
            stack2.append(v)
            if G.graph_dict.get(v) != None:
                for (v,w) in G.graph_dict[v]:
                    if G.explored[w] == False:
                        stack.append(w)            
    for i in reversed(stack2):
        t += 1
        G.finishing_time[i] = t

In [24]:
def kosaraju_two_pass_algorithm(filename):
    G_dict, G_rev_dict,n = read_input(filename)
    G_rev = Graph(n,G_rev_dict)
    
    #Run DFS loop on G_rev to compute "magical ordering" of nodes
    DFS_loop(G_rev)
    G_new_dict = {}
    
    #Processing nodes in decreasing order of finishing times
    for k,v in G_dict.items():
        new_k = G_rev.finishing_time[k]
        new_v = [(G_rev.finishing_time[i],G_rev.finishing_time[j]) for (i,j) in v]
        G_new_dict[new_k] = new_v
    G_new = Graph(n,G_new_dict)
    
    #Run DFS loop on G to dicover the SCC one by one
    DFS_loop(G_new)
    del G_new.leader[0]
    return Counter(G_new.leader).most_common(5)
    

In [25]:
start_time = time.time()
counter = kosaraju_two_pass_algorithm('week5_test1.txt') #3,3,3,0,0
print(counter)
print("--- %s seconds ---" % (time.time() - start_time))

[(6, 3), (4, 3), (9, 3)]
--- 0.01040196418762207 seconds ---


In [26]:
start_time = time.time()
counter = kosaraju_two_pass_algorithm('week5_test2.txt') #3,3,2,0,0
print(counter)
print("--- %s seconds ---" % (time.time() - start_time))

[(3, 3), (6, 3), (8, 2)]
--- 0.0052318572998046875 seconds ---


In [27]:
start_time = time.time()
counter = kosaraju_two_pass_algorithm('week5_test3.txt') #3,3,1,1,0
print(counter)
print("--- %s seconds ---" % (time.time() - start_time))

[(3, 3), (7, 3), (4, 1), (8, 1)]
--- 0.0021820068359375 seconds ---


In [28]:
start_time = time.time()
counter = kosaraju_two_pass_algorithm('week5_test4.txt') #7,1,0,0,0
print(counter)
print("--- %s seconds ---" % (time.time() - start_time))

[(8, 7), (4, 1)]
--- 0.0020029544830322266 seconds ---


In [29]:
start_time = time.time()
counter = kosaraju_two_pass_algorithm('week5_test5.txt') #6,3,2,1,0
print(counter)
print("--- %s seconds ---" % (time.time() - start_time))

[(12, 6), (5, 3), (6, 2), (2, 1)]
--- 0.009901046752929688 seconds ---


In [16]:
start_time = time.time()
read_input('week5.txt')
print("--- %s seconds ---" % (time.time() - start_time))

--- 28.60492181777954 seconds ---


In [30]:
start_time = time.time()
counter = kosaraju_two_pass_algorithm('week5.txt') #434821,968,459,313,211
print(counter)
print("--- %s seconds ---" % (time.time() - start_time))

[(615986, 434821), (617403, 968), (798411, 459), (161539, 313), (709991, 211)]
--- 47.82071900367737 seconds ---
